## Core Functions
- Report the range of times for each event for the given data set (done)
- Report the median time for each for the selected data set (done)
- Report fastest for each event along with that person and the rest of their time (EX "Christina Hepper has the fastest Swim at _____ and had the following other times...., Jamie Fend had the fastest T1 at _____ and had teh following other times...." (done, but need to improve formatting of result)
- Create functions
-- function to convert to integer minutes (done)
-- function for performance standardizing (done)
- Compute the fastest theoretical time (done)
- Compute the median theoretical time
- Report how selected Bib Number did relative to lead (EX "989 was 0.9x at swimming, 1.1x at T1....")
- Create a line plot with each event on the X, the Standardized Time on the Y (standardize to fastest?) and each line being an athlete, highlight the user and the median line

## Future
- Use mongo db to store data
- Automatically scrape the results
- Compute a "competitiveness" score for each race (like FSAE world: https://fs-world.org/)
- Calculate the percentile for eahc event for the user

In [50]:
import pandas as pd
import numpy as np
import pandas_bokeh
pandas_bokeh.output_notebook()
from bokeh.plotting import figure, show
pd.set_option("precision", 2)
events = ['Swim Minutes', 'T1 Minutes', 'Bike Minutes', 'T2 Minutes', 'Run Minutes']

Loading BokehJS ...

In [3]:
data = pd.read_excel('santa_cruz_data.xlsx', header = 0, index_col=None)

In [4]:
data.head()

,Bib,Name,Team Name,City,Start Time,Swim,T1,Bike,T2,Run,Chip Elapsed,Division,Div Place,Age,Age Place,Gender,Gender Place
0,1039,CHRISTINA HEPPER,NaN,MORGAN HILL,09:45:00,00:14:00,00:04:15,00:36:02,00:01:20,00:23:40,01:19:20,SPRINT-AGE GROUP,25.0,53.0,O/A,F,1.0
1,1006,JAMIE FEND,SANTA CRUZ TRIATHLON ASSOCIATION,APTOS,09:45:00,00:15:14,00:04:14,00:38:08,00:01:20,00:22:48,01:21:46,SPRINT-AGE GROUP,32.0,46.0,O/A,F,2.0
2,1112,ALEXANDRA LUZI,NaN,SAN FRANCISCO,09:45:00,00:12:50,00:06:41,00:38:04,00:01:48,00:22:44,01:22:08,SPRINT-AGE GROUP,35.0,30.0,O/A,F,3.0
3,1030,SUMI HACHMANN,TEEM SHEEPER,MENLO PARK,09:45:00,00:14:16,00:04:40,00:38:36,00:01:11,00:23:56,01:22:40,SPRINT-AGE GROUP,37.0,39.0,1,F,4.0
4,946,DENISE BUTLER,NaN,MOUNTAIN VIEW,09:45:00,00:12:34,00:04:37,00:40:08,00:00:49,00:24:46,01:22:56,SPRINT-AGE GROUP,39.0,34.0,1,F,5.0


In [5]:
#drop the columns we dont want (Team name)
#run only once
del data['Team Name']

In [6]:
#create a function to convert the time to integer seconds
#To create a new column, use the [] brackets with the new column name at the left side of the assignment.

def convertTime (time):
    timeMinutes = (time.hour*60)+(time.minute)+(time.second/60)
    return timeMinutes

#convert to integers
data["Swim Minutes"] = data["Swim"].apply(convertTime)
data["T1 Minutes"] = data["T1"].apply(convertTime)
data["Bike Minutes"] = data["Bike"].apply(convertTime)
data["T2 Minutes"] = data["T2"].apply(convertTime)
data["Run Minutes"] = data["Run"].apply(convertTime)
#data["Elapsed Minutes"] = data["Chip Elapsed"].apply(convertTime)

#get a sample time
#testTime = data.loc[0, "T1"]
#print(convertTime(testTime))

We need to convert the DataTime objects contatined in the data frame to integer minutes to do all of our nice math. Use the function and create new columsn to hold the data

In [7]:
summary = data.describe()
data.describe()

,Bib,Div Place,Age,Gender Place,Swim Minutes,T1 Minutes,Bike Minutes,T2 Minutes,Run Minutes
count,26.00,25.00,25.00,25.00,26.00,26.00,26.00,26.00,26.00
mean,1121.38,66.48,38.08,13.00,14.41,5.39,40.58,1.59,25.75
std,114.42,25.21,10.37,7.36,1.47,1.03,2.44,0.48,2.06
min,913.00,25.00,24.00,1.00,11.85,4.23,36.03,0.82,22.73
25%,1032.25,43.00,29.00,7.00,13.52,4.62,39.20,1.29,24.12
50%,1110.00,65.00,36.00,13.00,14.29,5.17,40.26,1.50,25.27
75%,1219.00,91.00,46.00,19.00,15.15,5.75,41.93,1.86,27.25
max,1292.00,104.00,60.00,25.00,18.30,8.07,47.55,2.97,30.08


In [8]:
data.head()

,Bib,Name,City,Start Time,Swim,T1,Bike,T2,Run,Chip Elapsed,...,Div Place,Age,Age Place,Gender,Gender Place,Swim Minutes,T1 Minutes,Bike Minutes,T2 Minutes,Run Minutes
0,1039,CHRISTINA HEPPER,MORGAN HILL,09:45:00,00:14:00,00:04:15,00:36:02,00:01:20,00:23:40,01:19:20,...,25.0,53.0,O/A,F,1.0,14.00,4.25,36.03,1.33,23.67
1,1006,JAMIE FEND,APTOS,09:45:00,00:15:14,00:04:14,00:38:08,00:01:20,00:22:48,01:21:46,...,32.0,46.0,O/A,F,2.0,15.23,4.23,38.13,1.33,22.80
2,1112,ALEXANDRA LUZI,SAN FRANCISCO,09:45:00,00:12:50,00:06:41,00:38:04,00:01:48,00:22:44,01:22:08,...,35.0,30.0,O/A,F,3.0,12.83,6.68,38.07,1.80,22.73
3,1030,SUMI HACHMANN,MENLO PARK,09:45:00,00:14:16,00:04:40,00:38:36,00:01:11,00:23:56,01:22:40,...,37.0,39.0,1,F,4.0,14.27,4.67,38.60,1.18,23.93
4,946,DENISE BUTLER,MOUNTAIN VIEW,09:45:00,00:12:34,00:04:37,00:40:08,00:00:49,00:24:46,01:22:56,...,39.0,34.0,1,F,5.0,12.57,4.62,40.13,0.82,24.77


In [21]:
#normalize the time for plotting
#select the data
dfPrescale = data[["Swim Minutes", "T1 Minutes", "Bike Minutes", "T2 Minutes", "Run Minutes"]]
dfPrescale

,Swim Minutes,T1 Minutes,Bike Minutes,T2 Minutes,Run Minutes
0,14.00,4.25,36.03,1.33,23.67
1,15.23,4.23,38.13,1.33,22.80
2,12.83,6.68,38.07,1.80,22.73
3,14.27,4.67,38.60,1.18,23.93
4,12.57,4.62,40.13,0.82,24.77
5,14.52,4.43,36.22,1.38,26.67
6,13.50,4.48,39.00,1.68,24.80
7,13.57,4.35,39.05,1.22,27.15
8,14.13,5.65,39.87,2.97,23.27
9,11.85,5.30,42.60,1.45,24.78


In [40]:
normalized_df = ((dfPrescale - dfPrescale.min())/(dfPrescale.max()-dfPrescale.min()))*10

#rename the columns
normalized_df.rename(columns = {"Swim Minutes":"Swim Scaled", "T1 Minutes":"T1 Scaled", "Bike Minutes":"Bike Scaled", 
                                "T2 Minutes":"T2 Scaled", "Run Minutes":"Run Scaled"}, inplace = True)
normalized_df

#merge to the main frame

merged = pd.concat([data, normalized_df], axis=1)

merged.head()

,Bib,Name,City,Start Time,Swim,T1,Bike,T2,Run,Chip Elapsed,...,Swim Minutes,T1 Minutes,Bike Minutes,T2 Minutes,Run Minutes,Swim Scaled,T1 Scaled,Bike Scaled,T2 Scaled,Run Scaled
0,1039,CHRISTINA HEPPER,MORGAN HILL,09:45:00,00:14:00,00:04:15,00:36:02,00:01:20,00:23:40,01:19:20,...,14.00,4.25,36.03,1.33,23.67,3.33,0.04,0.00,2.40,1.27
1,1006,JAMIE FEND,APTOS,09:45:00,00:15:14,00:04:14,00:38:08,00:01:20,00:22:48,01:21:46,...,15.23,4.23,38.13,1.33,22.80,5.25,0.00,1.82,2.40,0.09
2,1112,ALEXANDRA LUZI,SAN FRANCISCO,09:45:00,00:12:50,00:06:41,00:38:04,00:01:48,00:22:44,01:22:08,...,12.83,6.68,38.07,1.80,22.73,1.52,6.39,1.77,4.57,0.00
3,1030,SUMI HACHMANN,MENLO PARK,09:45:00,00:14:16,00:04:40,00:38:36,00:01:11,00:23:56,01:22:40,...,14.27,4.67,38.60,1.18,23.93,3.75,1.13,2.23,1.71,1.63
4,946,DENISE BUTLER,MOUNTAIN VIEW,09:45:00,00:12:34,00:04:37,00:40:08,00:00:49,00:24:46,01:22:56,...,12.57,4.62,40.13,0.82,24.77,1.11,1.00,3.56,0.00,2.77


In [45]:
reduced = merged.drop(merged.columns[[2,3,4,5,6,7,8,9,10,11,13,14]], axis=1)
reduced.head()

,Bib,Name,Age,Gender Place,Swim Minutes,T1 Minutes,Bike Minutes,T2 Minutes,Run Minutes,Swim Scaled,T1 Scaled,Bike Scaled,T2 Scaled,Run Scaled
0,1039,CHRISTINA HEPPER,53.0,1.0,14.00,4.25,36.03,1.33,23.67,3.33,0.04,0.00,2.40,1.27
1,1006,JAMIE FEND,46.0,2.0,15.23,4.23,38.13,1.33,22.80,5.25,0.00,1.82,2.40,0.09
2,1112,ALEXANDRA LUZI,30.0,3.0,12.83,6.68,38.07,1.80,22.73,1.52,6.39,1.77,4.57,0.00
3,1030,SUMI HACHMANN,39.0,4.0,14.27,4.67,38.60,1.18,23.93,3.75,1.13,2.23,1.71,1.63
4,946,DENISE BUTLER,34.0,5.0,12.57,4.62,40.13,0.82,24.77,1.11,1.00,3.56,0.00,2.77


In [46]:
# can send each runner data to a numnpy array that is named for that runner

In [59]:
reduced.plot_bokeh(kind = 'bar',
                  x='Name',
                  y=["Swim Minutes", "T1 Minutes"])

Figure(id='1956', ...)

In [46]:
#assign user variable
user = "SAMANTHA DEWEES"
index = data.index[data['Name']==user].tolist()
data.loc[index, ["Name", "Age", "Gender Place", "Swim Minutes", "T1 Minutes", "Bike Minutes", "T2 Minutes", "Run Minutes"]]

#data.loc[[user], ["Bib", "Name", "City", "Div Place", "Age"]]
#[data["Name"]=="SAMANTHA DEWEES"
#data.loc[data["Name"]=="SAMANTHA DEWEES"]

,Name,Age,Gender Place,Swim Minutes,T1 Minutes,Bike Minutes,T2 Minutes,Run Minutes
12,SAMANTHA DEWEES,25.0,13.0,15.03,7.07,39.65,2.22,23.63


In [10]:
def getMinTime (stage):
    minTime = data[stage].min()
    minIndex = data[stage].idxmin()
    #print("minTime is", minTime)
    #print("minIndex is", minIndex)
    return minTime, minIndex
    
def printResults (minIndex):
    #need to make this so it doesnt print reduntant colums or prints something nicely formatted
    print(data.iloc[minIndex,:])

In [11]:
shortestTime = 0
for stage in events:
    minTime, minIndex = getMinTime(stage)
    print("the fastest for ", stage)
    printResults(minIndex)
    shortestTime = shortestTime + minTime

print("the shortest possible time is: ", shortestTime)
#need to work on formatting

the fastest for  Swim Minutes
Bib                         1288
Name            CAROLE YOSHIWARA
City                   SUNNYVALE
Start Time              09:45:00
Swim                    00:11:51
T1                      00:05:18
Bike                    00:42:36
T2                      00:01:27
Run                     00:24:47
Chip Elapsed            01:26:02
Division        SPRINT-AGE GROUP
Div Place                   52.0
Age                         33.0
Age Place                      2
Gender                         F
Gender Place                10.0
Swim Minutes               11.85
T1 Minutes                   5.3
Bike Minutes                42.6
T2 Minutes                  1.45
Run Minutes                24.78
Name: 9, dtype: object
the fastest for  T1 Minutes
Bib                         1006
Name                  JAMIE FEND
City                       APTOS
Start Time              09:45:00
Swim                    00:15:14
T1                      00:04:14
Bike                    00:3

In [12]:
#Swim_min =data["Swim Minutes"].min() 
#T1_min = data["T1 Minutes"].min()
#Bike_min = data["Bike Minutes"].min()
#T2_min = data["T2 Minutes"].min()
#Run_min = data["Run Minutes"].min()

#shortestTime = Swim_min+T1_min+Bike_min+T2_min+Run_min

#print(shortestTime)